In [1]:
# default_exp update

# Calculating Carbon Intensity

<br>

### Imports

In [36]:
#exports
import pandas as pd
import numpy as np
import json

import dask
import dask.dataframe as dd

import matplotlib.pyplot as plt
import seaborn as sns

import os
from ipypb import track
from warnings import warn
import FEAutils as hlp

In [2]:
from IPython.display import JSON

In [6]:
# Loading
df_emissions = pd.read_csv('../data/raw/pi/pi_emissions.csv').set_index('authorisation id / permit id')

# Checking
assert (df_emissions<0).sum().sum() == 0, 'Negative emissions are present'

df_emissions.head()

,2013,2014,2015,2016,2017,2018,2019
authorisation id / permit id,,,,,,,
AP3130LY,NaN,NaN,NaN,NaN,NaN,2.274672e+08,2.586962e+08
AP3233LU,2.448600e+07,1.270700e+07,1.522300e+07,1.969800e+07,1.300200e+07,1.535400e+07,2.153750e+07
AP3238ZU,1.293660e+08,4.596400e+07,7.200000e+07,NaN,NaN,NaN,NaN
AP3330LB,1.100363e+10,9.221626e+09,NaN,NaN,NaN,NaN,NaN
AP3630LG,6.873370e+08,1.116502e+09,1.073794e+09,1.314658e+09,1.484210e+09,1.686550e+09,1.079406e+09


In [7]:
#exports
def get_B1610_columns(data_dir):
    B1610_files = [f for f in os.listdir(data_dir) if '.csv' in f]

    columns = []

    for B1610_file in track(B1610_files, label='columns'):
        df_B1610_week = pd.read_csv(f'{data_dir}/{B1610_file}')
        columns += list(df_B1610_week.columns)

    columns = ['datetime'] + sorted(list(set(columns)-set(['datetime'])))

    return columns

In [8]:
B1610_dir = '../data/raw/elexon'
B1610_columns = get_B1610_columns(B1610_dir)

B1610_columns[:5]

        columns [█████████████████████████████████████████████] 288/288 [00:21<00:00, 0.07s/it]

['datetime', 'ABRBO-1', 'ABRTW-1', 'ABTH7', 'ABTH7G']

In [9]:
#exports
@dask.delayed
def read_B1610_file(filename, columns):
    df_B1610_week = pd.read_csv(filename)
    cols_to_add = list(set(columns) - set(df_B1610_week.columns))
    df_B1610_week[cols_to_add] = np.NaN
    df_B1610_week = df_B1610_week[columns]
    
    return df_B1610_week

def load_B1610_dask_stream_df(data_dir, dt_col='datetime', columns=None):
    # Identifying columns
    if columns is None:
        columns = get_B1610_columns(data_dir)
    
    # Loading data
    B1610_files = [f for f in os.listdir(data_dir) if '.csv' in f]
    df_B1610 = dd.from_delayed([read_B1610_file(f'{data_dir}/{B1610_file}', columns) for B1610_file in B1610_files])

    # Formatting date index
    if dt_col is not None:
        df_B1610[dt_col] = df_B1610[dt_col].map(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True))
        df_B1610 = df_B1610.set_index(dt_col)
    
    return df_B1610

In [10]:
%%time

ddf_B1610 = load_B1610_dask_stream_df(B1610_dir, columns=B1610_columns)
df_B1610 = ddf_B1610.compute()

df_B1610.head()

Wall time: 2min 30s


,ABRBO-1,ABRTW-1,ABTH7,ABTH7G,ABTH8,ABTH8G,ABTH9,ABTH9G,ACHRW-1,AKGLW-2,...,WILCT-1,WLNYO-2,WLNYO-3,WLNYO-4,WLNYW-1,WTMSO-1,WYLF-1,WYLF-2,WYLF-3,WYLF-4
datetime,,,,,,,,,,,,,,,,,,,,,
2015-10-04 23:00:00+00:00,NaN,NaN,421.692,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,132.90,NaN,NaN,95.602,9.858,227.380,212.410,NaN,NaN
2015-10-04 23:30:00+00:00,NaN,NaN,425.096,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,143.02,NaN,NaN,107.324,14.890,227.470,212.530,NaN,NaN
2015-10-05 00:00:00+00:00,NaN,NaN,423.292,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,158.98,NaN,NaN,118.342,23.074,227.666,212.720,NaN,NaN
2015-10-05 00:30:00+00:00,NaN,NaN,431.148,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,163.18,NaN,NaN,140.900,36.890,227.826,212.844,NaN,NaN
2015-10-05 01:00:00+00:00,NaN,NaN,418.772,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,160.86,NaN,NaN,148.838,47.772,228.054,213.086,NaN,NaN


In [11]:
with open('../data/mappings/ngc_bmu_id_to_pi_permits.json', 'r') as fp:
    ngc_bmu_id_to_pi_permits = json.load(fp)

JSON([ngc_bmu_id_to_pi_permits])

<IPython.core.display.JSON object>

In [12]:
with open('../data/mappings/ngc_bmu_id_to_fuel.json', 'r') as fp:
    ngc_bmu_id_to_fuel = json.load(fp)

JSON([ngc_bmu_id_to_fuel])

<IPython.core.display.JSON object>

In [14]:
s_ngc_bmu_id_to_fuel = pd.Series(ngc_bmu_id_to_fuel)
potential_emitting_ngc_bmu_ids = s_ngc_bmu_id_to_fuel[~s_ngc_bmu_id_to_fuel.isin(['wind', 'npshyd', 'nuclear', 'ps', 'hydro'])].index

print(f'There are {len(potential_emitting_ngc_bmu_ids)} NGC BMU ids that are potentially carbon emitting')

There are 252 NGC BMU ids that are potentially carbon emitting


In [15]:
print(f'There are {len(ngc_bmu_id_to_pi_permits)} NGC BMU ids that have been mapped to a PI permit')

There are 131 NGC BMU ids that have been mapped to an eutl account


In [16]:
ngc_bmu_ids_in_B1610_without_fuel_mapping = sorted(list(set(df_B1610.columns) - set(ngc_bmu_id_to_fuel.keys())))

assert len(ngc_bmu_ids_in_B1610_without_fuel_mapping) == 0, f'The following NGC BMU ids are associated with power plants in the B1610 dataframe but do not have an associated fuel type:\n{", ".join(ngc_bmu_ids_in_B1610_without_fuel_mapping)}'

In [25]:
flatten_list = lambda t: [item for sublist in t for item in sublist]
account_ids_not_in_emissions_df = sorted(list(set(flatten_list([[elem] if isinstance(elem, str) else elem for elem in ngc_bmu_id_to_pi_permits.values()])) - set(df_emissions.index)))

assert len(account_ids_not_in_emissions_df) == 0, f'The following account ids are associated with power plants but do not have an entry in the emissions dataframe:\n{", ".join(account_ids_not_in_emissions_df)}'

In [38]:
ngc_bmu_ids_not_in_B1610_df = sorted(list(set(ngc_bmu_id_to_pi_permits.keys()) - set(potential_emitting_ngc_bmu_ids)))

if len(ngc_bmu_ids_not_in_B1610_df) > 0:
    warn(f'The following account ids are associated with carbon emitting power plants but do not have an entry in the emissions dataframe:\n{", ".join(ngc_bmu_ids_not_in_B1610_df)}')

<ipython-input-38-75a9fa3e44d5>:4: UserWarning: The following account ids are associated with carbon emitting power plants but do not have an entry in the emissions dataframe:
DIDC1, DIDC2, DIDC4
  warn(f'The following account ids are associated with carbon emitting power plants but do not have an entry in the emissions dataframe:\n{", ".join(ngc_bmu_ids_not_in_B1610_df)}')


In [107]:
relevant_ngc_bmu_ids = sorted(list(set(potential_emitting_ngc_bmu_ids) & set(B1610_columns) & set(ngc_bmu_id_to_pi_permits.keys())))
s_ngc_bmu_id_to_pi_permits = pd.Series(ngc_bmu_id_to_pi_permits).loc[relevant_ngc_bmu_ids]
s_ngc_bmu_id_to_pi_permit_groups = s_ngc_bmu_id_to_pi_permits.apply(lambda x: '__'.join(x) if isinstance(x, list) else x)
unique_accounts_ids = sorted(list(set(flatten_list([[elem] if isinstance(elem, str) else elem for elem in s_ngc_bmu_id_to_pi_permits.values]))))

df_power = pd.DataFrame()

for accounts_id in track(unique_accounts_ids):
    ngc_bmu_ids_associated_with_account = list(s_ngc_bmu_id_to_pi_permits[s_ngc_bmu_id_to_pi_permit_groups.str.contains(accounts_id)].index)
    df_power[accounts_id] = df_B1610[ngc_bmu_ids_associated_with_account].sum(axis=1)

df_power.head()

 [████████████████████████████████████████████████████████████] 53/53 [00:01<00:00, 0.02s/it]

,AP3233LU,AP3330LB,AP3630LG,AP3633BL,BJ8022IZ,BK0701IW,BL6217IM,BP3239LA,BS5380IC,BS8192IV,...,VP3538XX,VP3930LH,VP3933RJ,WP3135JL,WP3535LZ,XP3038SW,YP3030LR,YP3133LL,YP3930LZ,ZP3133LM
datetime,,,,,,,,,,,,,,,,,,,,,
2015-10-04 23:00:00+00:00,0.0,0.0,0.0,0.0,444.8,0.0,0.0,0.0,0.0,0.0,...,1130.7,0.0,0.0,306.128,122.758,271.898,0.0,0.0,558.6,0.0
2015-10-04 23:30:00+00:00,0.0,0.0,0.0,0.0,307.7,0.0,0.0,0.0,0.0,0.0,...,1141.0,0.0,0.0,301.646,122.498,271.498,0.0,0.0,543.2,0.0
2015-10-05 00:00:00+00:00,0.0,0.0,0.0,0.0,235.4,0.0,0.0,0.0,0.0,0.0,...,1063.8,0.0,0.0,406.684,122.618,271.498,0.0,0.0,518.9,0.0
2015-10-05 00:30:00+00:00,0.0,0.0,0.0,0.0,231.5,0.0,0.0,0.0,0.0,0.0,...,1160.3,0.0,0.0,497.890,122.778,271.398,0.0,0.0,517.8,0.0
2015-10-05 01:00:00+00:00,0.0,0.0,0.0,0.0,231.7,0.0,0.0,0.0,0.0,0.0,...,1023.2,0.0,0.0,478.622,122.838,271.398,0.0,0.0,503.3,0.0


In [108]:
df_annual_power = df_power['2016':'2020'].resample('Y').sum()/2 # /2 to convert MW to MWh
df_annual_power.index = df_annual_power.index.year

df_annual_power

,AP3233LU,AP3330LB,AP3630LG,AP3633BL,BJ8022IZ,BK0701IW,BL6217IM,BP3239LA,BS5380IC,BS8192IV,...,VP3538XX,VP3930LH,VP3933RJ,WP3135JL,WP3535LZ,XP3038SW,YP3030LR,YP3133LL,YP3930LZ,ZP3133LM
datetime,,,,,,,,,,,,,,,,,,,,,
2016,30345.647,2764546.682,3504873.90,2367985.1,7721634.800,4650643.65,5640833.3,0.577,4149322.700,1844939.505,...,10066096.25,2242735.437,3789.680,1640353.872,724544.619,2155750.666,3512.710,1802497.265,6135281.05,20917.358
2017,19760.419,2564488.810,3937387.75,3326434.1,6761155.150,3560728.95,5590636.8,0.040,2627556.800,1900711.768,...,7440541.70,1029404.575,10488.662,3175527.997,996464.952,2465171.001,2410.535,2295307.750,6466648.85,20066.480
2018,9754.161,3562229.695,4278755.59,2416697.9,7288425.483,2778260.50,5015982.1,0.005,1981281.400,1167262.796,...,7906913.15,539389.063,37558.558,2822464.797,941954.050,921114.370,5147.745,2533506.125,6026352.20,9773.992
2019,6047.750,744093.294,2770691.91,1936933.8,7627097.975,4474174.55,4437811.7,347561.653,3228100.902,1154560.551,...,9374445.35,0.000,66879.244,1746648.845,865416.017,184489.403,5729.452,2032307.860,6377714.00,6727.073
2020,16974.632,459573.883,2135379.22,2073032.3,7237835.541,3446762.85,4211782.6,1431336.141,1907092.600,982491.471,...,5167114.90,0.000,51679.886,0.000,974412.886,740140.039,3008.054,2018613.490,4645229.70,3109.117


In [109]:
grouped_pi_permits = list(s_ngc_bmu_id_to_pi_permit_groups[s_ngc_bmu_id_to_pi_permit_groups.str.contains('__')].drop_duplicates().str.split('__').values)

print(f'There are {len(grouped_pi_permits)} installations which are associated with multiple PI permits')

There are 9 installations which are associated with multiple PI permits


In [112]:
df_grouped_emissions = pd.DataFrame()
df_grouped_power = pd.DataFrame()

non_grouped_permits = list(s_ngc_bmu_id_to_pi_permit_groups[~s_ngc_bmu_id_to_pi_permit_groups.str.contains('__')].unique())
df_grouped_emissions[non_grouped_permits] = df_emissions.T[non_grouped_permits]
df_grouped_power[non_grouped_permits] = df_annual_power[non_grouped_permits]

for pi_permits_group in grouped_pi_permits:
    group_name = '__'.join(pi_permits_group)
    s_group_emissions = df_emissions.T[pi_permits_group].sum(axis=1)
    df_grouped_emissions[group_name] = s_group_emissions
    
    df_grouped_power[group_name] = df_annual_power[pi_permits_group].sum(axis=1)
    
df_grouped_emissions.index = df_grouped_emissions.index.astype(int)

df_grouped_emissions.head(3)

,ZP3133LM,RP3438GG,LP3833LM,EP3833LY,YP3030LR,YP3930LZ,VP3530LS,VP3930LH,WP3535LZ,VP3337SR,...,CP3035MK,LP3631SL__NP3033RD,SP3535LT__WP3135JL,DP3933DN__NP3634WE__VP3133LP,CP3537SM__NP3833RC,EP3533RY__RP3432SG,HP3733LJ__NP3633RK,VP3933RJ__SP3233LQ,EP3133RZ__AP3330LB,DP3433DM__EP3934WC__XP3038SW
2013,10146200.0,NaN,115686000.0,288000000.0,1.737800e+09,9.047780e+08,2.311890e+10,1.149590e+10,14100000.0,8.235000e+09,...,NaN,280212840.0,1.020000e+10,1.985180e+09,290187000.0,2.065142e+09,6.561910e+08,108000000.0,1.100363e+10,192302700.0
2014,9642300.0,NaN,154300000.0,326000000.0,NaN,1.274951e+09,2.374538e+10,7.799832e+09,NaN,2.935200e+09,...,1.840000e+09,357057780.0,8.880000e+09,1.792559e+09,130359000.0,2.070438e+09,1.537248e+09,138695220.0,9.221626e+09,143622400.0
2015,7548240.0,NaN,126200000.0,682000000.0,NaN,1.568566e+09,2.343064e+10,4.748503e+09,NaN,2.037000e+09,...,2.371371e+09,227356000.0,6.780000e+09,1.805591e+09,177848480.0,1.600883e+09,1.854300e+09,26439000.0,4.769920e+09,561487000.0


In [122]:
s_emissions

2016    827660500.0
2017    940187560.0
2018    346663100.0
2019     68210580.0
Name: DP3433DM__EP3934WC__XP3038SW, dtype: float64

In [126]:
df_annual_carbon_intensity = pd.DataFrame()
s_carbon_intensity = pd.Series(index=df_grouped_emissions.columns, dtype='float64')

for account_id in df_grouped_emissions.columns:
    s_emissions = df_grouped_emissions.loc[2016:2020, account_id] 
    s_power = df_grouped_power.loc[2016:2020, account_id]
        
    df_annual_carbon_intensity[account_id] = s_emissions/s_power
    
    weights = s_power.loc[df_annual_carbon_intensity[account_id].dropna().index]
    
    if len(weights) > 0:
        s_carbon_intensity[account_id] = np.average(df_annual_carbon_intensity[account_id].dropna(), weights=weights)

df_annual_carbon_intensity = df_annual_carbon_intensity.replace(np.inf, np.nan)

df_annual_carbon_intensity.head()

,ZP3133LM,RP3438GG,LP3833LM,EP3833LY,YP3030LR,YP3930LZ,VP3530LS,VP3930LH,WP3535LZ,VP3337SR,...,CP3035MK,LP3631SL__NP3033RD,SP3535LT__WP3135JL,DP3933DN__NP3634WE__VP3133LP,CP3537SM__NP3833RC,EP3533RY__RP3432SG,HP3733LJ__NP3633RK,VP3933RJ__SP3233LQ,EP3133RZ__AP3330LB,DP3433DM__EP3934WC__XP3038SW
2016,678.925608,370.669547,NaN,394.145065,NaN,376.681521,888.656362,935.907092,NaN,901.816271,...,377.952614,176.155953,484.651522,129.486862,192.596961,179.659625,NaN,0.000000,484.689229,127.977153
2017,717.995882,365.217156,499.406692,399.693278,NaN,376.516965,885.235854,985.359911,NaN,NaN,...,377.463083,186.994568,466.064227,132.659850,195.113383,180.679860,NaN,0.000000,477.375255,127.129458
2018,1757.751592,382.466584,570.197507,422.983762,NaN,385.441959,908.192236,1002.853111,NaN,NaN,...,392.347294,187.228320,513.735371,135.216235,205.427272,185.825018,NaN,333.559132,467.760263,125.450618
2019,3045.908674,365.366357,501.968635,394.316501,NaN,375.521699,896.072663,NaN,NaN,NaN,...,383.247772,184.369651,395.042199,131.630250,196.240798,183.696429,NaN,330.723258,489.550844,123.242092
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df_ngc_bmu_emissions = pd.DataFrame({
    'fuel': s_ngc_bmu_id_to_fuel.loc[s_ngc_bmu_id_to_pi_permit_groups.index],
    'gco2_per_kWh': s_ngc_bmu_id_to_pi_permit_groups.map(s_carbon_intensity),
    'pi_permits': s_ngc_bmu_id_to_pi_permit_groups
})

df_ngc_bmu_emissions.index.name = 'ngc_bmu_id'

df_ngc_bmu_emissions.head()

,fuel,gco2_per_kWh,pi_permits
ngc_bmu_id,,,
BRGG-1,ccgt,1152.985854,ZP3133LM
CARR-1,ccgt,371.062909,RP3438GG
CARR-2,ccgt,371.062909,RP3438GG
CDCL-1,ccgt,183.960365,LP3631SL__NP3033RD
CORB-1,ccgt,533.404114,LP3833LM


In [129]:
# want to find eutl_account groups that have only a single fuel type
# then report the value_counts of the pure types

In [130]:
s_pi_permits_fuel_types = df_ngc_bmu_emissions.groupby('pi_permits')['fuel'].unique()
s_pi_permits_pure_fuel_types = s_pi_permits_fuel_types[s_pi_permits_fuel_types.apply(len)==1].apply(lambda x: x[0])

s_pi_permits_pure_fuel_types.value_counts()

ccgt       28
coal        3
ocgt        1
biomass     1
Name: fuel, dtype: int64

In [ ]:
# need to focus on the permits rather than ngc when plotting

In [144]:
ocgt_permits = s_pi_permits_pure_fuel_types[s_pi_permits_pure_fuel_types=='coal'].index

df_filtered_emissions = df_ngc_bmu_emissions[df_ngc_bmu_emissions['pi_permits'].isin(ocgt_permits)]

df_filtered_emissions

,fuel,gco2_per_kWh,pi_permits
ngc_bmu_id,,,
COTPS-1,coal,470.431765,SP3535LT__WP3135JL
COTPS-2,coal,470.431765,SP3535LT__WP3135JL
COTPS-3,coal,470.431765,SP3535LT__WP3135JL
COTPS-4,coal,470.431765,SP3535LT__WP3135JL
EGGPS-1,coal,958.737013,VP3930LH
EGGPS-2,coal,958.737013,VP3930LH
EGGPS-3,coal,958.737013,VP3930LH
EGGPS-4,coal,958.737013,VP3930LH
IRNPS-2,coal,NaN,HP3733LJ__NP3633RK


In [140]:
missing_relevant_bmus_from_B1610 = sorted(list((set(potential_emitting_ngc_bmu_ids) & set(B1610_columns)) - set(df_ngc_bmu_emissions.index)))

pct_missing = len(missing_relevant_bmus_from_B1610)/len(set(potential_emitting_ngc_bmu_ids) & set(B1610_columns))

0.2695035460992908

In [151]:
df_ngc_bmu_emissions.to_csv('../data/intermediate/pi_first_estimate.csv')